# **Practical 2**

In [8]:
# Install required packages
!pip install requests

In [9]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [10]:
# Read data from CoinGecko API
target = "zar"
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd," + target
response = requests.get(url)
data = response.json()

data


{'bitcoin': {'usd': 114304, 'zar': 2018611}}

In [11]:
# Extract USD price and apply transformation
usd_price = data["bitcoin"]["usd"]
target_price = data["bitcoin"][target]
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in USD: {usd_price}")
print(f"Estimated price in Target currency: {target_price}")


Bitcoin price in USD: 114304
Estimated price in Target currency: 2018611


In [12]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    -- These are SQL comments. Similarly /* ... */ for multi-line comments
    -- If timestamp should be unique (e.g., one price per moment):
    -- timestamp TEXT PRIMARY KEY,
    -- If multiple prices per second are expected, or if you want a simple unique ID:
    -- id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    usd REAL,
    target REAL
)
''')

# Insert the new row
cursor.execute("INSERT INTO prices VALUES (?, ?, ?)", (timestamp, usd_price, target_price))
conn.commit()
conn.close()


In [13]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()


('2025-08-19T14:31:38.013872', 114304.0, 2018611.0)
